# Optimisation – Project 1

## Part 3.2
Median problem with weighted Euclidean distance

\begin{equation}
\min_{x \in \mathbb{R}} \sum_{i \in \mathcal{M}} v^i d_2(a^i, x)
\end{equation}

where $\mathcal{M} = \{1, \dots, m\}$ and $0 ≤ v^i \in \mathbb{R}, \; \forall i \in \mathcal{M}$.

We will here look into the Weiszfeld algorithm for solving the problem above.

## See overleaf for some of the solutions

## 4

In [2]:
import numpy as np

## A solution for implementing an apropriate stopping criterion

In [9]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import ConvexHull, convex_hull_plot_2d   # get a convex hull computation algorithm
 
 
 
#points = np.random.rand(10, 2)   # 30 random points in 2-D
#hull = ConvexHull(points)
 
#vecs = []
#for i in range(len(hull.vertices)):
 ##   element = points[hull.vertices[i]]
 #   element = element.tolist()
 #   vecs.append(element)
 
def distance(x_1,x_2):
    x_1 = np.array(x_1)
    x_2 = np.array(x_2)
    diff = x_1 - x_2
    return np.linalg.norm(diff, 2) 
 
 
def isCounterClockwise(n1,n2, n3):
    return ((n1 <n2 <n3) or (n3 < n1 < n2) or (n2 < n3 < n1))
 
def Area(vtxList, n1, n2, n3): # get area of triangle made of point 1,2, 3 from a list of vertices
    n = len(vtxList)
    vtx1 = vtxList[n1]
    vtx2 = vtxList[n2]
    vtx3 = vtxList[n3]
    # print(vtx1, vtx2, vtx3)
    a = distance(vtx1,vtx2)
    b = distance(vtx3,vtx2)
    c = distance(vtx1,vtx3)
    s = (a + b + c) / 2
    area = (s*(s-a)*(s-b)*(s-c)) ** 0.5
    if isCounterClockwise(n1,n2,n3):
        return round(area,3)
    else:
        return round(-area,3)
 
 
def GetAllAntiPodalPairs(vtxList):
    pairList = []
    n = len(vtxList)
    i_1 = 1
    i_n = 0
    i = i_n
    j = (i + 1)%n
    while (Area(vtxList,i, (i + 1)%n, (j + 1)%n) > Area(vtxList, i, (i + 1)%n, j)):
        j = (j + 1)%n
        j0 = j
    while (j != i_1):
        i = (i + 1)%n
        pairList.append([i, j])
        while ((Area(vtxList, i, (i + 1)%n, (j + 1)%n) > Area(vtxList, i, (i + 1)%n, j))):
            j = (j + 1)%n
            if (j != i_1):
                pairList.append([i, j])
        if (Area(vtxList, i, (i + 1)%n, (j + 1)%n) == Area(vtxList, i, (i + 1)%n, j)):
            if ((i, j) != (j0, i_n)):
                pairList.append([i, (j + 1)%n])
 
    return(pairList)
 
 
 
def findDiameter(polygon):
    maxx = 0
    antiPodalPairs = GetAllAntiPodalPairs(polygon)
    for pair in antiPodalPairs:
        pairPoint1 = polygon[pair[0]]
        pairPoint2 = polygon[pair[1]]
        candidate = distance(pairPoint1, pairPoint2)
        if (candidate > maxx):
            maxx = candidate
    return maxx
 
 
# EXAMPLE
 
polygonZ = [[0,0], [3,0], [2,1], [0,1]]
print(findDiameter(polygonZ))

3.1622776601683795
[0, 1]


In [76]:
def ourWeiszfeld(A, v, epsilon = 10**(-10)):
    """
    A = [a^1,..., a^i,..., a^m]^T   where   a^i = [a_1^i, a_2^i]
    Hence, A is a m times 2 matrix
    v = [v^1,..., v^i,..., v^m]^T   where   0<v^i \in \mathbb{R}, \forall i \in \mathcal{M} 
    epsilon > 0
    """
    m = len(A) # len(A) gives number of rows i.e. m
    
    # We define a test function to make the code leaner
    def testK(k):
        sum1 = 0
        sum2 = 0
        for i in range(m): 
            if i != k:
                dist = np.linalg.norm((A[k], A[i])) # d_2(*,*) euclidean distance
                sum1 += v[i]* (A[k].item(0) - A[i].item(0))/(dist)
                sum2 += v[i]* (A[k].item(1) - A[i].item(1))/(dist)
            
            # else: # This "else" might be unecasarry?
              #  continue
                
        result = np.sqrt(sum1**2 + sum2**2)
        return (result < v[k])
    
    for k in range(m): 
        # theorem 1 test if the theorem is fulfilled for a "k"
        if testK(k): # do we need to pass the k? should rather not?
            x_star = A[k] # could just return A[k], but this is clearer
    
    # choose a starting point x = [x_1, x_2]^T, can be found solving the median problem with ||\cdot||_2^2
    x = np.ones(A.shape[1])
    
    i = 0
    while i < 100:
        i += 1
        for j in range(0, len(x)):
            enum = 0
            deno = 0
            for i in range(m):
                print(np.asarray(A[i]))
                print(x)
                dist = np.linalg.norm((A[i], x))
                enum += v[i]*A[i].item(j)/dist
                deno += v[i]/dist

            x[j] = enum/deno
        
    return x

In [77]:
ourWeiszfeld(A, v)

[[1 2]]
[1. 1.]


ValueError: shapes (1,2) and (1,2) not aligned: 2 (dim 1) != 1 (dim 0)

## Gradient descent algorithm

In [5]:
def steepestDescent(x_init, obj_func, max_iter = 10000, threshold = 10**(-10)):
    # We create the gradient function for the objective function using autograd
    gradient = grad(obj_func)
    
    # Here we initialize
    i = 0
    x = np.array(x_init) 
    rho = 0.4
    c = 0.6
    diff = np.full((len(x), 1), 100) # initialize some value
    while i < max_iter and diff.any() > threshold: # Can use same stopping criterion as above to measure time performance
        i += 1
        a = 1
        grad_k = gradient(x)
        # I make a pk for convenience 
        pk = -grad_k
        
        # The following is backtracking
        while obj_func(x + a*pk) > obj_func(x) + a*c*np.transpose(grad_k) @ pk: # i.e. repeat until not true
            a = rho*a
        
        # The new x is stored
        x = x + a*pk
        diff = abs(a*pk)
    return x

In [23]:
# Here are just tests
A = np.matrix('1 2 ; 3 4 ; 5 6 ; 7 8 ; 9 10 ; 11 12')
print(A.item((1, 1)))
print(A[1].item(1))
len(A[0])
print(A[1] @ np.transpose(A[2]))
b = np.full(3, 9)
print(b)

4
4
[[39]]
[9 9 9]


In [25]:
v = np.random.rand(6)

In [73]:

ourWeiszfeld(A, v)

ValueError: shapes (1,2) and (1,2) not aligned: 2 (dim 1) != 1 (dim 0)

## 5

Compare using precision, speed and possibly stability
